# Multi-cycle image registration

It is often advantageous to acquire data in multiple cycles, in which different objects are stained in different imaging cycles. A key step in preprocessing these kinds of data is image registration. This is implemented in ``blimp`` using functions in ``blimp.preprocessing.registration``. This notebook demonstrates the use of these functions for correcting image data.

Note: You must run ``notebooks/0_setup.ipynb`` first to specify the configuration and download the test data.

In [1]:
import napari

from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import logging
import skimage

from blimp.data import load_example_data
from blimp.log import configure_logging
from blimp.constants import blimp_config
from blimp.preprocessing.registration import (
    calculate_shifts,
    apply_shifts,
    register_2D,
    transform_2D,
    TransformationParameters
)
from aicsimageio import AICSImage
from pathlib import Path

configure_logging(verbosity=2)
# ensure that example data is downloaded
load_example_data()
# read correct blimp_config -- created with 0_setup.ipynb
blimp_config.config_fname = "blimp.ini"
print(blimp_config)


24-Jan-23 21:50:18 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


Reading config from /Users/scottberry/source/blimp/notebooks/blimp.ini
unpacked_dir = /Users/scottberry/source/blimp/notebooks/_data/raw
archive_path = /Users/scottberry/source/blimp/notebooks/_data/archive/_data.zip
Reading config from blimp.ini
BLIMPConfig (fname: blimp.ini)
EXPERIMENT_DIR: /Users/scottberry/source/blimp/notebooks/_experiments
BASE_DATA_DIR: /Users/scottberry/source/blimp/notebooks/_data/raw
data_config/exampledata: /Users/scottberry/source/blimp/notebooks/ExampleData_constants.py



Load some images from the examples directory using the ``aicsimageio`` package

In [2]:
cyc01 = AICSImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_01' / 'r05c03f15-fk1fl1-mip.ome.tiff')
cyc02 = AICSImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_02' / 'r05c03f15-fk1fl1-mip.ome.tiff')

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


In [3]:
cyc01

<AICSImage [Reader: OmeTiffReader, Image-is-in-Memory: False]>

In [4]:
print(cyc01.channel_names)
print(cyc01.dims)

['DAPI', 'Alexa 568']
<Dimensions [T: 1, C: 2, Z: 1, Y: 2160, X: 2160]>


get some input data

In [5]:
dapi01 = cyc01.get_image_data("YX",C=0)
dapi02 = cyc02.get_image_data("YX",C=0)

generate a rotated version of dapi02 and crop dapi01 to match size

# multi-cycle data

Data acquired on the operetta in a real two-cycle experiment. 

In [15]:
viewer = napari.view_image(dapi01,colormap="red",blending="additive",name="Cycle 1 - uncorrected")
viewer.add_image(dapi02,colormap="green",blending="additive",name="Cycle 2 - uncorrected")

<Image layer 'Cycle 2 - uncorrected' at 0x7fa4a43e6130>

Align cycle 2 to cycle 1

In [8]:
settings = TransformationParameters('rigid')
dapi02_corrected, parameters_cyc01_cyc02 = register_2D(fixed=dapi01, moving=dapi02, settings=settings)

In [9]:
viewer.add_image(dapi02_corrected,colormap="blue",blending="additive",name="Cycle 2 - corrected")

<Image layer 'Cycle 2 - corrected' at 0x7fa495c96880>

In [11]:
viewer2 = napari.view_image(dapi01_crop,colormap="red",blending="additive",name="Cycle 2 - raw")
viewer2.add_image(dapi02_rotated_crop,colormap="green",blending="additive",name="Cycle 1 - rotated")

<Image layer 'Cycle 1 - rotated' at 0x7fa45063bdc0>

## simulated rotations

Try with the simulated data that has been rotated

In [33]:
dapi02_rotated = skimage.transform.rotate(dapi02,3,preserve_range=True).astype(np.uint16)
# remove outer 100 pixels
dapi02_rotated_crop = dapi02_rotated[100:-100,100:-100]
dapi01_crop = dapi01[100:-100,100:-100]

viewer2 = napari.view_image(dapi01_crop,colormap="red",blending="additive",name="Cycle 1 - original")
viewer2.add_image(dapi02_rotated_crop,colormap="green",blending="additive",name="Cycle 2 - rotated")

<Image layer 'Cycle 2 - rotated' at 0x7fa496d6c190>

Test first using translation only

In [34]:
dapi02_rotated_crop_registered, translation_parameters = register_2D(
    fixed=dapi01_crop,
    moving=dapi02_rotated_crop,
    settings=TransformationParameters(transformation_mode='translation')
)

viewer2.add_image(dapi02_rotated_crop_registered,colormap="blue",blending="additive",name="Cycle 1 - registered (translation)")

<Image layer 'Cycle 1 - registered (translation)' at 0x7fa496d6cb20>

now allow rotations in addition to translation

In [36]:
dapi02_rotated_crop_registered_rigid, rigid_parameters = register_2D(
    fixed=dapi01_crop,
    moving=dapi02_rotated_crop,
    settings=TransformationParameters(transformation_mode='rigid')
)

viewer2.add_image(dapi02_rotated_crop_registered_rigid,colormap="blue",blending="additive",name="Cycle 1 - registered (rotation)")

<Image layer 'Cycle 1 - registered (rotation)' at 0x7fa496d6c130>